## Package

In [34]:
import numpy as np
import os
import argparse
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn 

## Config

In [35]:
parser = {
    "root": "../",
    "seeds": 123,
    "batch_size": 16,
    "class_a_index": 5, # 5 -> dog
    "class_b_index": 3, # 3 -> cat
    "class_a_size": None, # Use None to read all class_a samples (5000 rows)
    "class_b_size": None,
    "epoch": 10,
    "download_cifar10": True, # set to True if you have not downloaded cifar10 dataset 
}
config = argparse.Namespace(**parser)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Reprodicibility

In [36]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    
set_seed(config.seeds)

## Load Data

In [37]:
class CustomDataLoader:
    
    def __init__(self, class_a_size=None, class_b_size=None, seeds=123, download=True):
        """
        Arguments:
            class_a_size: The number of samples for class a (e.g. dog).
            class_b_size: The number of samples for class b (e.g. cat).
        """
        self.class_a_size = class_a_size # e.g. 5000
        self.class_b_size = class_b_size # e.g. 5000
        self.seeds = seeds
        self.classes = ['plane', 'car', 'bird', 'cat',
                   'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
        self.download = download
        
    def sampling(self, indices, num, seeds=123):
        """
        Sample num samples from indices randomly. If num is a float and 0.0 <= num <= 1.0,
        we sample num percentage of samples from indices.
        """
        np.random.seed(self.seeds)
        if isinstance(num, float) and 0 <= num <= 1:
            size = int(num * len(indices))
            samples = np.random.choice(indices, size)
        elif isinstance(num, int) and num <= len(indices):
            size = num
            samples = np.random.choice(indices, size)
        elif num is None:
            samples = indices
        else:
            print("Please make sure 'num' is in the correct range")
        return samples       
        
    def get_subset_index(self, labels, targets):
        
        # get subsets
        sample_indices = []
        for i in range(len(labels)):
            if labels[i] in targets:
                sample_indices.append(i)
            
        return sample_indices

    def load_dataset(self, data_dir, batch_size, train_a_label=5, train_b_label=3):
        
        transform = transforms.Compose(
            [transforms.ToTensor(),
             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        
        # 1. read train data belonged to train_labels
        trainset = torchvision.datasets.CIFAR10(root=data_dir, train=True,
                                                download=self.download, transform=transform)
        
        # 1-1. get index belonged to class a and b with sampling
        class_a_indices = self.get_subset_index(trainset.targets, [train_a_label])
        class_a_indices = self.sampling(class_a_indices, self.class_a_size, self.seeds)
        
        class_b_indices = self.get_subset_index(trainset.targets, [train_b_label])
        class_b_indices = self.sampling(class_b_indices, self.class_b_size, self.seeds)
        
        train_index_subset = np.concatenate((class_a_indices, class_b_indices))
        trainsubset = torch.utils.data.Subset(trainset, train_index_subset)

        # 1. read test data and separate them into (dog, cat) and (the other 8)
        testset = torchvision.datasets.CIFAR10(root=data_dir, train=False,
                                               download=self.download, transform=transform)
        test_index_class_ab = self.get_subset_index(
            labels=testset.targets,
            targets=[train_a_label, train_b_label]
        )
        test_index_others = self.get_subset_index(
            labels=testset.targets,
            targets=[cls for cls in range(len(self.classes)) if cls not in [train_a_label, train_b_label]]
        )
        testsubset_ab = torch.utils.data.Subset(testset, test_index_class_ab)
        testsubset_others = torch.utils.data.Subset(testset, test_index_others)

        return trainsubset, (testsubset_ab, testsubset_others)

    def generate_loader(self, dataset, batch_size, shuffle=True, drop_last=True):
        dataloader = torch.utils.data.DataLoader(
            dataset,
            batch_size=batch_size,
            shuffle=True,
            drop_last=True
        )
        return dataloader

In [38]:
mydataloader = CustomDataLoader(
    class_a_size=config.class_a_size,
    class_b_size=config.class_b_size,
    seeds=config.seeds,
    download=config.download_cifar10)

trainset, (testset_ab, testset_others) = mydataloader.load_dataset(
    data_dir=os.path.join(config.root, "data"),
    batch_size=config.batch_size,
    train_a_label=config.class_a_index, # dog
    train_b_label=config.class_b_index, # cat
)

train_loader = mydataloader.generate_loader(
    dataset=trainset,
    batch_size=config.batch_size,
    shuffle=True,
    drop_last=True,
)

test_ab_loader = mydataloader.generate_loader(
    dataset=testset_ab,
    batch_size=config.batch_size,
    shuffle=False,
    drop_last=False,
)

test_others_loader = mydataloader.generate_loader(
    dataset=testset_others,
    batch_size=config.batch_size,
    shuffle=False,
    drop_last=False,
)

Files already downloaded and verified
Files already downloaded and verified


## Model Architecture

### CNN Based

In [39]:
class CNNBased(nn.Module):
    def __init__(self, ):
        super(CNNBased, self).__init__()
        
    def forward(self, inputs):
        pass

### ResNet

In [40]:
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)

In [41]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out


In [42]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

## Train Test Functions

### Train

In [43]:
def train(trainloader, criterion, optimizer):
    total_step = len(trainloader)
    num_epochs = config.epoch
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(trainloader):
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i+1) % 100 == 0:
                print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

In [44]:
model = ResNet(ResidualBlock, [2, 2, 2]).to(device)
criterion = nn.CrossEntropyLoss()
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
train(train_loader, criterion, optimizer)

Epoch [1/10], Step [100/625] Loss: 0.7109
Epoch [1/10], Step [200/625] Loss: 0.6873
Epoch [1/10], Step [300/625] Loss: 0.7250
Epoch [1/10], Step [400/625] Loss: 0.8079
Epoch [1/10], Step [500/625] Loss: 0.6152
Epoch [1/10], Step [600/625] Loss: 0.5968
Epoch [2/10], Step [100/625] Loss: 0.6955
Epoch [2/10], Step [200/625] Loss: 0.6989
Epoch [2/10], Step [300/625] Loss: 0.6648
Epoch [2/10], Step [400/625] Loss: 0.7299
Epoch [2/10], Step [500/625] Loss: 0.5812
Epoch [2/10], Step [600/625] Loss: 0.6880
Epoch [3/10], Step [100/625] Loss: 0.5172
Epoch [3/10], Step [200/625] Loss: 0.6711
Epoch [3/10], Step [300/625] Loss: 0.6042
Epoch [3/10], Step [400/625] Loss: 0.5409
Epoch [3/10], Step [500/625] Loss: 0.5610
Epoch [3/10], Step [600/625] Loss: 0.7007
Epoch [4/10], Step [100/625] Loss: 0.5880
Epoch [4/10], Step [200/625] Loss: 0.4590
Epoch [4/10], Step [300/625] Loss: 0.5548
Epoch [4/10], Step [400/625] Loss: 0.4427
Epoch [4/10], Step [500/625] Loss: 0.6561
Epoch [4/10], Step [600/625] Loss:

### Test

In [45]:
def evaluation(testloader):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in testloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

In [46]:
evaluation(test_ab_loader)

Accuracy of the model on the test images: 78.15 %


### Utils

- Save anything for plots?

## Start Training

### Reload Data

### Training Requirements

- optimizer
- criterion

### Start Here